從《Python 網路爬蟲與資料分析入門實戰》第三章的範例中練習爬蟲

從第三章GOOGLE搜尋股價爬蟲範例中練習自己寫的程式碼

書中原始程式碼來源：https://github.com/jwlin/web-crawler-tutorial/tree/master/ch3

# 壹、寫爬蟲函數：(1)抓取內容(2)重複抓取

In [1]:
import requests
from bs4 import BeautifulSoup

def google_stock(stid):
    #1.向網站發出請求
        ##1.1google搜尋網址(後面須填入股票代碼)
    url = 'https://www.google.com/search?q='
    
        ##1.2User-Agent訊息
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                        'AppleWebKit/537.36 (KHTML, like Gecko) '
                        'Chrome/66.0.3359.181 Safari/537.36'}
    
        ##1.3發出請求(填入網址與headers)
    resp = requests.get(url+stid, headers=headers)
    
    #2.剖析網站原始碼
    soup = BeautifulSoup(resp.text, 'html5lib')
    
    #3.定位資訊的大概位置：g-card-section
    section = soup.find_all('g-card-section')
    #4.創造空的清單(放爬下來的資料)
    data=[]

    #5.爬下我所需要的資料並存入清單
        #5.1創建一個空的字典(放資料)
    stock = dict()
    
        #5.2創造「名稱」:div標籤名稱oPhL2e屬性的字串
    stock['name'] = soup.find('div','oPhL2e').text
    
        #5.3創造「及時價格」:第2個g-card-section中span的字串
    stock['current_price'] = section[1].find('span').text
    
        #5.4創造「收盤時間」:第2個g-card-section中第五個div的span
    stock['time'] = section[1].find_all('div')[4].find('span').text
    
        #5.5創造「current_change」:第2個g-card-section中第五個div的第四個span
    stock['current_change'] =section[1].find_all('div')[4].find_all('span')[3].text

        #5.6使用迴圈去爬資料與資料名稱欄位
            #5.6.1在第四個g-card-section中的所有tr標籤找
    for a in section[3].find_all('tr'):
            #5.6.2名稱欄位:第一個td的字串
        key=a.find_all('td')[0].text
            #5.6.3資料:第二個td的字串
        value=a.find_all('td')[1].text
            #5.6.4將每筆資料都轉化成[欄位]與資料
        stock[key] = value

            #5.6.5將資料放入清單中並返回結果
    data.append(stock)    
    return data


    #6.重複抓取資料並儲存清單
def repeat_google_stock(idlist):
        #6.1創一個新的空清單
    data2=[]
    
        #6.2使用迴圈抓取不同股票資料
    for i in idlist:
        
        #6.3使用函數取得資料
        data=google_stock(i)
        
        #6.4將資料加入空的清單中
        data2 += data
        
        #6.5返回結果
    return data2

    #7.取得資料:輸入股票idlist使用函數
idlist=['TPE:2330','NASDAQ:GOOG','NYSE:T']
data=repeat_google_stock(idlist)

# 貳、處理資料並儲存

In [2]:
import pandas as pd
import numpy as np  

    #8.處理資料並儲存
        #8.1創建欄位資料
name=['name','市值','current_price','time','current_change',
      '上次收盤價','開盤','最高','最低','本益比','殖利率',
      '52 週高點','52 週低點']

        #8.2將資料轉成dataframe(輸入欄位資料)
test=pd.DataFrame(columns=name,data=data)

        #8.3整理資料中的索引(從1開始編號)
test.index = np.arange(1,len(test)+1)
    
        #8.4將索引重新命名成編號
test.index.names = ['stock_NO.']
    
        #8.5印出結果
print(test)

        #8.6儲存成csv檔案
test.to_csv('stock.csv',encoding='utf_8_sig')

                           name        市值 current_price  \
stock_NO.                                                 
1                      台灣積體電路製造     8.19兆    316.00 TWD   
2          Alphabet Inc Class C  9241.66億  1,340.62 USD   
3                          AT&T  2790.51億     38.20 USD   

                                   time current_change     上次收盤價        開盤  \
stock_NO.                                                                    
1          已收盤: 12月9日 下午1:30 [GMT+8] ·            免責聲明    313.00    314.00   
2            已收盤: 12月6日 下午7:19 [EST] ·            免責聲明  1,328.13  1,333.44   
3            已收盤: 12月6日 下午7:58 [EST] ·            免責聲明     38.19     38.29   

                 最高        最低    本益比    殖利率    52 週高點  52 週低點  
stock_NO.                                                      
1            316.50    314.00  24.89  2.69%    339.50  206.50  
2          1,344.00  1,333.44  29.17      -  1,344.00  970.11  
3             38.58     38.14  17.12  5.34%     39.70   2

# 無註解程式碼

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np  

def google_stock(stid):
    url = 'https://www.google.com/search?q='
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                        'AppleWebKit/537.36 (KHTML, like Gecko) '
                        'Chrome/66.0.3359.181 Safari/537.36'}
    
    resp = requests.get(url+stid, headers=headers)
    soup = BeautifulSoup(resp.text, 'html5lib')
    section = soup.find_all('g-card-section')

    data=[]
    stock = dict()

    stock['name'] = soup.find('div','oPhL2e').text
    stock['current_price'] = section[1].find('span').text
    stock['time'] = section[1].find_all('div')[4].find('span').text
    stock['current_change'] =section[1].find_all('div')[4].find_all('span')[3].text

    for a in section[3].find_all('tr'):
        key=a.find_all('td')[0].text
        value=a.find_all('td')[1].text
        stock[key] = value

    data.append(stock)    
    return data




def repeat_google_stock(idlist):
    data2=[]

    for i in idlist:
        data=google_stock(i)
        data2 += data
        
    return data2



idlist=['TPE:2330','NASDAQ:GOOG','NYSE:T']
data=repeat_google_stock(idlist)




name=['name','市值','current_price','time','current_change',
      '上次收盤價','開盤','最高','最低','本益比','殖利率',
      '52 週高點','52 週低點']
test=pd.DataFrame(columns=name,data=data)
test.index = np.arange(1,len(test)+1)
test.index.names = ['stock_NO.']
    
print(test)
test.to_csv('stock.csv',encoding='utf_8_sig')

# urllib套件版本的爬蟲

In [ ]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np  

def google_stock(stid):
    url = 'https://www.google.com/search?q='
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                        'AppleWebKit/537.36 (KHTML, like Gecko) '
                        'Chrome/66.0.3359.181 Safari/537.36'}
    
    request = urllib.request.Request(url+stid, headers=headers)
    resp = urllib.request.urlopen(request).read()
    soup = BeautifulSoup(resp, 'html5lib')
    section = soup.find_all('g-card-section')

    data=[]
    stock = dict()

    stock['name'] = soup.find('div','oPhL2e').text
    stock['current_price'] = section[1].find('span').text
    stock['time'] = section[1].find_all('div')[4].find('span').text
    stock['current_change'] =section[1].find_all('div')[4].find_all('span')[3].text

    for a in section[3].find_all('tr'):
        key=a.find_all('td')[0].text
        value=a.find_all('td')[1].text
        stock[key] = value

    data.append(stock)    
    return data




def repeat_google_stock(idlist):
    data2=[]

    for i in idlist:
        data=google_stock(i)
        data2 += data
        
    return data2



idlist=['TPE:2330','NASDAQ:GOOG','NYSE:T']
data=repeat_google_stock(idlist)




name=['name','市值','current_price','time','current_change',
      '上次收盤價','開盤','最高','最低','本益比','殖利率',
      '52 週高點','52 週低點']
test=pd.DataFrame(columns=name,data=data)
test.index = np.arange(1,len(test)+1)
test.index.names = ['stock_NO.']
    
print(test)
test.to_csv('stock.csv',encoding='utf_8_sig')